In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np


In [ ]:
## load the trained model, scaler pickle, onehot
model = load_model('model.h5')

## load the encoder and scaler
with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('onehot_encoder.pkl', 'rb') as f:
    onehot_encoder_geo = pickle.load(f)

In [ ]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [ ]:
## One-hot encode the Geography
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [ ]:
input_df = pd.DataFrame([input_data])
input_df

In [ ]:
## encode categorical variables
input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])
input_df

In [ ]:
## concatination one hot encoded
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)

In [ ]:
## Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

In [ ]:
## Predict churn
prediction = model.predict(input_scaled)
prediction

In [ ]:
prediction_proba = prediction[0][0]

In [ ]:
prediction_proba

In [ ]:
if prediction_proba > 0.5:
    print("The customer is likely to churn with a probability of {:.2f}%".format(prediction_proba * 100))
else:
    print("The customer is unlikely to churn with a probability of {:.2f}%".format((1 - prediction_proba) * 100))